## 1 - Initializing

In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt

In [ ]:
gpus= tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Image preparation

In [ ]:
import cv2
import imghdr

In [ ]:
img_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [ ]:
data_set_dir = 'new_data_set'
os.listdir(data_set_dir)

In [ ]:
for image_class in os.listdir(data_set_dir):
    for image in os.listdir(os.path.join(data_set_dir, image_class)):
        image_path = os.path.join(data_set_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            # plt.imshow(img)
            # plt.show()
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print('image ext not supported{}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('issue with image{}'.format(image_path))

### Creating dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory('new_data_set')

In [ ]:
dataset_iterator = dataset.as_numpy_iterator()

In [ ]:
batch = dataset_iterator.next()

In [ ]:
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx,img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

## 2 - Preprocessing dataset

## scaling

In [ ]:
#scaling our data using .map(lambda.....)
scaled_dataset = dataset.map(lambda x, y: (x/255, y))

In [ ]:
scaled_dataset_iterator = scaled_dataset.as_numpy_iterator()
scaled_dataset_batch = scaled_dataset_iterator.next()
scaled_dataset_batch[0].max()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx,img in enumerate(scaled_dataset_batch[0][:4]):
    ax[idx].imshow(img.astype(float))
    ax[idx].title.set_text(scaled_dataset_batch[1][idx])

## splitting data

In [ ]:
len(scaled_dataset)

In [ ]:
train_size = int(len(scaled_dataset)*0.7)
val_size = int(len(scaled_dataset)*0.2)
test_size = int(len(scaled_dataset)*0.1)

In [ ]:
train = scaled_dataset.take(train_size)
val = scaled_dataset.skip(train_size).take(val_size)
test = scaled_dataset.skip(train_size+val_size).take(test_size)

In [ ]:
# len(val)

## 3 - Building the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16,(3,3),1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(16,(3,3),1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile('adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

## 4 - Training the model

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# print(val)

In [ ]:
# list(train)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

## 5 - visualization

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

## 6 - Evaluating the model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator():
    x, y = batch
    yhat = model.predict(x)
    yhat_flat = [np.argmax(i) for i in yhat]
    pre.update_state(y, yhat_flat)
    re.update_state(y, yhat_flat)
    acc.update_state(y, yhat_flat)

In [ ]:
print(f'Precision:{pre.result().numpy()},Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

## 7 - testing the model